In [2]:
#This Kernal plots 500 mb heights and vorticity from a specified GFS simulation (must be within the last week)
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
from datetime import datetime
from datetime import timedelta 
from wrf import to_np


#Date to be grabbed from the NOAA NOMADS Server: http://nomads.ncep.noaa.gov:9090/dods/
year=2018
month=12
day=27
#times=20
level1=500
def run(sim):
    if sim < 10:
        run="0{0}".format(sim)
        return run
    else: 
        run="{0}".format(sim)
        return run
sim=12
run=run(sim)
model='gfs'

def rgb(r,g,b):
    return '#%02x%02x%02x' % (r, g, b)
def getColor(val,rng,col1,col2):
    r1,g1,b1 = col1
    r2,g2,b2 = col2

    rdif = float(r2 - r1)
    gdif = float(g2 - g1)
    bdif = float(b2 - b1)
    
    r3 = r2 + (-1.0 * val * (rdif / float(rng)))
    g3 = g2 + (-1.0 * val * (gdif / float(rng)))
    b3 = b2 + (-1.0 * val * (bdif / float(rng)))

    return rgb(r3,g3,b3)
    
#Returns a list of colors matching up with the range(s) provided
def gradient(*args):
    
    #Initialize an empty color list
    colors = []
    
    #Loop through the passed lists, following a format of ['#0000FF','#00FFFF',5]
    #where [0] = start color, [1] = end color, [2] = number of colors in range
    for val in args:
        end_color = val[0].replace("#","")
        start_color = val[1].replace("#","")
        range_color = val[2]-1
        
        #Convert start and end values to RGB
        start_rgb = tuple(ord(c) for c in start_color.decode('hex'))
        end_rgb = tuple(ord(c) for c in end_color.decode('hex'))
            
        #Loop through the number of colors to add into the lit
        for x in range(0,range_color+1):

            #Get hex value for the color at this point in the range
            colors.append(getColor(x,range_color,start_rgb,end_rgb))
            
    #Return the list of colors
    return colors

#Plotting intervals 
slevs=np.arange(450, 640, 6)
vlevs=np.arange(10,70,1)

#Specified Color Gradient using gradient generator functions. 
#Color codes can be found/made here: http://www.perbang.dk/rgbgradient/
colors = gradient(['FFFD0A', 'FF7114', 20], ['FF6F14', 'FF0515', 20], ['FA0010', '560006', 20])
colors = matplotlib.colors.ListedColormap(colors)

#Function that grabs GFS run based on given date
def file(year, month, day):
    if month <= 9 and day <=9:
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}0{2}0{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    elif month > 9 and day <= 9: 
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}{2}0{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    elif month <= 9 and day > 9:
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}0{2}{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    else:    
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}{2}{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
infile=file(year, month, day)
data=xr.open_dataset(infile)
lats=data.lat
lons1=data.lon
nlats = len(lats)
nlons = len(lons1)
lons, lats = np.meshgrid(lons1, lats)



#Basemap is used to specify area to be plotted/porjection to be used 
m = Basemap(width=5120000,height=3500000, lat_1=49.5,lat_2=29.5,lat_0=39.5,lon_0=-98.3, rsphere=(6378137.00,6356752.3142), projection='lcc',resolution = 'h', area_thresh = 500)
x, y = m(lons, lats)
times= np.arange(0, 64, 2)



#Loop that plots for each forecast time 
for i in times:
    hgt1=(data.hgtprs.isel(time=i).sel(lev=level1))*.1
    vort=(data.absvprs.isel(time=i).sel(lev=level1))*100000
    uwind=(data.ugrdprs.isel(time=i).sel(lev=level1))*1.94384
    vwind=(data.vgrdprs.isel(time=i).sel(lev=level1))*1.94384
    hr= i*3
    t2=timedelta(hours=hr)
    init=datetime(year, month, day, sim ) 
    datei= init.strftime("%a %b %d, %Y at %Hz")
    val=init + t2
    datev= val.strftime("%a %b %d, %Y at %Hz")
    u_rot, v_rot, x, y = m.rotate_vector(uwind, vwind, lons, lats, returnxy=True)
    plt.figure(figsize=(12, 8))
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    cax = m.contourf(x, y, vort, vlevs, cmap=colors)
    clb = m.colorbar(cax) 
    clb.ax.set_title('s^-1')
    #m.colorbar(cax,ticks=[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65])
    cs = m.contour(x,y,hgt1,levels=slevs,colors='k',linewidths=1.5)
    plt.barbs(x[::12,::12], y[::12,::12], u_rot[::12, ::12],
         v_rot[::12, ::12], length=6)
    plt.title("{0} mb Absolute Vorticity (s^-1), Geopotential Heights (Dm), and Winds (Kts)\n{1} hr Forecast".format(level1, hr), loc='left', fontsize=11)
    plt.title("Init: {0}\nValid: {1}".format(datei, datev), loc= 'right', fontsize=11)
    #plt.savefig("GFS_{0}{1}{2}{3}z_{4}_500mb".format(year, month, day, sim, hr), dpi=120, bbox_inches='tight')
    plt.show()




KeyboardInterrupt: 

In [1]:
#This Kernal plots 850 mb temperature

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
from datetime import datetime
from datetime import timedelta 
from wrf import to_np

year=2017
month=12
day=29
#times=20
level1=850
def run(sim):
    if sim < 10:
        run="0{0}".format(sim)
        return run
    else: 
        run="{0}".format(sim)
        return run
sim=12
run=run(sim)
model='gfs'

def rgb(r,g,b):
    return '#%02x%02x%02x' % (r, g, b)
def getColor(val,rng,col1,col2):
    r1,g1,b1 = col1
    r2,g2,b2 = col2

    rdif = float(r2 - r1)
    gdif = float(g2 - g1)
    bdif = float(b2 - b1)
    
    r3 = r2 + (-1.0 * val * (rdif / float(rng)))
    g3 = g2 + (-1.0 * val * (gdif / float(rng)))
    b3 = b2 + (-1.0 * val * (bdif / float(rng)))

    return rgb(r3,g3,b3)
    
#Returns a list of colors matching up with the range(s) provided
def gradient(*args):
    
    #Initialize an empty color list
    colors = []
    
    #Loop through the passed lists, following a format of ['#0000FF','#00FFFF',5]
    #where [0] = start color, [1] = end color, [2] = number of colors in range
    for val in args:
        end_color = val[0].replace("#","")
        start_color = val[1].replace("#","")
        range_color = val[2]-1
        
        #Convert start and end values to RGB
        start_rgb = tuple(ord(c) for c in start_color.decode('hex'))
        end_rgb = tuple(ord(c) for c in end_color.decode('hex'))
            
        #Loop through the number of colors to add into the lit
        for x in range(0,range_color+1):

            #Get hex value for the color at this point in the range
            colors.append(getColor(x,range_color,start_rgb,end_rgb))
            
    #Return the list of colors
    return colors
slevs=np.arange(90, 180, 3)
vlevs=np.arange(-40,40,1)
colors = gradient(['FFF5F8', 'E600BC', 10], ['BD009A', 'A88FC1', 10], ['9C78BF', '5D00B8', 10],['0016E6','00F0E8',10], ["00EB81","00990B",10]
                 ,["F0F401","F05600",10],["FA0000","F00089",10],["FF38AA","FED2EB",9])
colors = matplotlib.colors.ListedColormap(colors)
def file(year, month, day):
    if month <= 9 and day <=9:
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}0{2}0{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    elif month > 9 and day <= 9: 
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}{2}0{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    elif month <= 9 and day > 9:
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}0{2}{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    else:    
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}{2}{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
infile=file(year, month, day)
data=xr.open_dataset(infile)
lats=data.lat
lons1=data.lon
nlats = len(lats)
nlons = len(lons1)
lons, lats = np.meshgrid(lons1, lats)
m = Basemap(width=5120000,height=3500000, lat_1=49.5,lat_2=29.5,lat_0=39.5,lon_0=-98.3, rsphere=(6378137.00,6356752.3142), projection='lcc',resolution = 'h', area_thresh = 500)
x, y = m(lons, lats)
times= np.arange(0, 64, 2)
for i in times:
    uwind=(data.ugrdprs.isel(time=i).sel(lev=level1))*1.94384
    vwind=(data.vgrdprs.isel(time=i).sel(lev=level1))*1.94384
    hgt1=(data.hgtprs.isel(time=i).sel(lev=level1))*.1
    temp=(data.tmpprs.isel(time=i).sel(lev=level1))-273.15
    hr= i*3
    t2=timedelta(hours=hr)
    init=datetime(year, month, day, sim ) 
    datei= init.strftime("%a %b %d, %Y at %Hz")
    val=init + t2
    datev= val.strftime("%a %b %d, %Y at %Hz")
  
    u_rot, v_rot, x, y = m.rotate_vector(uwind, vwind, lons, lats, returnxy=True)
    plt.figure(figsize=(12, 8))
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    cax = m.contourf(x, y, temp, vlevs, cmap=colors)
    v = np.arange(-40, 40, 10)
    clb = m.colorbar(cax, ticks=v) 
    clb.ax.set_title('C')
    cs = m.contour(x,y,hgt1,levels=slevs,colors='k',linewidths=1.5)
    plt.barbs(x[::12,::12], y[::12,::12], u_rot[::12, ::12],
         v_rot[::12, ::12], length=6)
    plt.title("{0} mb Temperature (C), Geopotential Heights (Dm), and Winds (Kts)\nGFS-{1} hr Forecast".format(level1, hr), loc='left', fontsize=11)
    plt.title("Init: {0}\nValid: {1}".format(datei, datev), loc= 'right', fontsize=11)
    plt.savefig("GFS_{0}{1}{2}{3}z_{4}_{5}mb".format(year, month, day, sim, hr, level1), dpi=120, bbox_inches='tight')


C:\Users\wflam\Anaconda2\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
#This kernal plots 300 mb winds, thickness, and MSLP 

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
from datetime import datetime
from datetime import timedelta 
import scipy.ndimage as ndimage

year=2018
month=1
day=26
#times=20
level=300
level1=500
level2=1000
def run(sim):
    if sim < 10:
        run="0{0}".format(sim)
        return run
    else: 
        run="{0}".format(sim)
        return run
sim=12
run=run(sim)
model='gfs'
tlevs1=np.arange(468.,546.,6.)
tlevs2=np.arange(546,624,6.)
def rgb(r,g,b):
    return '#%02x%02x%02x' % (r, g, b)
def getColor(val,rng,col1,col2):
    r1,g1,b1 = col1
    r2,g2,b2 = col2

    rdif = float(r2 - r1)
    gdif = float(g2 - g1)
    bdif = float(b2 - b1)
    
    r3 = r2 + (-1.0 * val * (rdif / float(rng)))
    g3 = g2 + (-1.0 * val * (gdif / float(rng)))
    b3 = b2 + (-1.0 * val * (bdif / float(rng)))

    return rgb(r3,g3,b3)
    
#Returns a list of colors matching up with the range(s) provided
def gradient(*args):
    
    #Initialize an empty color list
    colors = []
    
    #Loop through the passed lists, following a format of ['#0000FF','#00FFFF',5]
    #where [0] = start color, [1] = end color, [2] = number of colors in range
    for val in args:
        end_color = val[0].replace("#","")
        start_color = val[1].replace("#","")
        range_color = val[2]-1
        
        #Convert start and end values to RGB
        start_rgb = tuple(ord(c) for c in start_color.decode('hex'))
        end_rgb = tuple(ord(c) for c in end_color.decode('hex'))
            
        #Loop through the number of colors to add into the lit
        for x in range(0,range_color+1):

            #Get hex value for the color at this point in the range
            colors.append(getColor(x,range_color,start_rgb,end_rgb))
            
    #Return the list of colors
    return colors
vlevs=np.arange(30,130,10)
colors = gradient(['31E6FC', '180189', 5], ['AF31FC', 'FFE6FD', 4])
colors = matplotlib.colors.ListedColormap(colors)
def file(year, month, day):
    if month <= 9 and day <=9:
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}0{2}0{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    elif month > 9 and day <= 9: 
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}{2}0{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    elif month <= 9 and day > 9:
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}0{2}{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
    else:    
        infile="http://nomads.ncep.noaa.gov:9090/dods/{0}_0p25/{0}{1}{2}{3}/{0}_0p25_{4}z".format(model, year, month, day, run)
        return infile
infile=file(year, month, day)
data=xr.open_dataset(infile)
lats=data.lat
lons1=data.lon
nlats = len(lats)
nlons = len(lons1)
lons, lats = np.meshgrid(lons1, lats)
m = Basemap(width=5120000,height=3500000, lat_1=49.5,lat_2=29.5,lat_0=39.5,lon_0=-98.3, rsphere=(6378137.00,6356752.3142), projection='lcc',resolution = 'h', area_thresh = 500)
x, y = m(lons, lats)
clevs=np.arange(900,1100,4) 
times= np.arange(0, 64, 2)    
for i in times:
    uwind=(data.ugrdprs.isel(time=i).sel(lev=level))
    vwind=(data.vgrdprs.isel(time=i).sel(lev=level))
    mslp=.01*(data.msletmsl.isel(time=i))
    hgt500=(data.hgtprs.isel(time=i).sel(lev=500))
    hgt1000=(data.hgtprs.isel(time=i).sel(lev=1000))
    windm=(uwind**2 + vwind**2)**.5
    thickness = ndimage.gaussian_filter((.1*(hgt500-hgt1000)), sigma=1.5, order=0)  

    hr= i*3
    t2=timedelta(hours=hr)
    init=datetime(year, month, day, sim ) 
    datei= init.strftime("%a %b %d, %Y at %Hz")
    val=init + t2
    datev= val.strftime("%a %b %d, %Y at %Hz")
    mslpf=ndimage.gaussian_filter(mslp,sigma=1.5,order=0)

    u_rot, v_rot, x, y = m.rotate_vector(uwind, vwind, lons, lats, returnxy=True)
    windm=(u_rot**2 + v_rot**2)**.5

    plt.figure(figsize=(12, 8))
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    cb = m.contour(x,y,thickness,tlevs1,colors='b', linestyles = 'dashed',linewidths=1.75)
    plt.clabel(cb,inline=1,inline_spacing=0,fontsize=10,fmt='%1.0f')
    cr = m.contour(x,y,thickness,tlevs2,colors='r', linestyles = 'dashed',linewidths=1.75)
    plt.clabel(cr,inline=1,inline_spacing=0,fontsize=10,fmt='%1.0f')
    v = np.arange(30, 130, 10)
    cs = m.contour(x, y, mslpf,clevs,colors='k',linewidths=1.75)
    plt.clabel(cs, inline=1, inline_spacing=0, fontsize=10, fmt = '%1.0f')
    cax = m.contourf(x, y, windm, vlevs, cmap=colors, alpha=.50)
    
    clb = m.colorbar(cax, ticks=v) 
    clb.ax.set_title('m/s')

    plt.barbs(x[::12,::12], y[::12,::12], u_rot[::12, ::12],
         v_rot[::12, ::12], length=5.9)
    plt.title("1000-500 mb Thickness (Dm), MSLP (hPa), and 300 mb Winds (m/s)\nGFS-{1} hr Forecast".format(level1, hr, level2), loc='left', fontsize=11)
    plt.title("Init: {0}\nValid: {1}".format(datei, datev), loc= 'right', fontsize=11)
    plt.savefig("GFS_{0}{1}{2}{3}z_{4}_500_1000_mb".format(year, month, day, sim, hr, level1, level2), dpi=120, bbox_inches='tight')


C:\Users\wflam\Anaconda2\lib\site-packages\matplotlib\contour.py:370: RuntimeWarning: invalid value encountered in true_divide
  dist = np.add.reduce(([(abs(s)[i] / L[i]) for i in range(xsize)]), -1)
C:\Users\wflam\Anaconda2\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<xarray.DataArray 'prmslmsl' (lat: 721, lon: 1440)>
array([[  998.791328,   998.791328,   998.791328, ...,   998.791328,
          998.791328,   998.791328],
       [  998.531328,   998.531328,   998.535313, ...,   998.529297,
          998.529297,   998.531328],
       [  998.169297,   998.169297,   998.171328, ...,   998.163359,
          998.165313,   998.167344],
       ..., 
       [ 1004.193359,  1004.191328,  1004.187344, ...,  1004.203359,
         1004.199297,  1004.197344],
       [ 1004.867344,  1004.865313,  1004.865313, ...,  1004.871328,
         1004.869297,  1004.869297],
       [ 1005.529297,  1005.529297,  1005.529297, ...,  1005.529297,
         1005.529297,  1005.529297]])
Coordinates:
    time     datetime64[ns] 2017-12-27T18:00:00
  * lat      (lat) float64 -90.0 -89.75 -89.5 -89.25 -89.0 -88.75 -88.5 ...
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 1.25 1.5 1.75 2.0 2.25 2.5 ...
